<a href="https://colab.research.google.com/github/rubyvanrooyen/observation_planning/blob/main/comet67P/67P_resolution_and_sensitivity_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from astropy import units as u
from astropy import constants as const

import numpy as np

In [ ]:
bandwidth = 54.*u.MHz
numchans = 32768
centerfreq = 1.6654*u.GHz

## Spectral resolution

Spectral resolution for MeerKAT wideband and spectral line modes = $\Delta v \approx \Delta f \frac{c}{f}$
with $\Delta v$ the velocity span (km/s), $\Delta f$ the channel width, $f$ the centre frequency and $c$ the speed of light in a vacuum.

In [ ]:
def velocity_resolution(bw, fc, nc):
    # channel width
    df = bw.to(u.Hz)/nc
    # channel velocity resolution
    dv = (df*const.c/fc.to(u.Hz)).to(u.km/u.s)
    print(dv)
    print(f'Bandwidth {bw} over {nc} channels for channel width {dv:.3f} at {fc}')

velocity_resolution(bw=bandwidth,
                    fc=centerfreq,
                    nc=numchans)

0.2966511030072308 km / s
Bandwidth 54.0 MHz over 32768 channels for channel width 0.297 km / s at 1.6654 GHz


## Spatial resolution

The FWHM of the synthesized beam (point spread function), which is the inverse Fourier transform of a (weighted) u-v sampling distribution.    
The resolution in arcsec can be approximated as: FWHM (") = 63 / max_baseline (km) / frequency (GHz)

In [ ]:
def spatial_resolution(fc,
                       bmax=7.7*u.km):
    wavelength = fc.to(u.nm, equivalencies=u.spectral())
    baseline = bmax.to(u.m)/wavelength.to(u.m)
    FWHMsb = 1.02/baseline * u.rad.to(u.arcsec) * u.arcsec
    print(f'MeerKAT Resolution @ {fc} = {FWHMsb:.3}')

spatial_resolution(fc=centerfreq)

MeerKAT Resolution @ 1.6654 GHz = 4.92 arcsec


## Point source sensitivity

Use radiometer equation to estimate detectability

The theoretical thermal rms intensity-sensitivity $\sigma_\mathrm{S}$ of a (naturally-weighted) image is found using 
\begin{align*}\sigma_\mathrm{S} = \frac{2 k_\mathrm{B} T_\mathrm{sys}}{A_\mathrm{eff} \sqrt{N_\mathrm{p} N_\mathrm{a}(N_\mathrm{a}-1) \Delta\nu \Delta t}}\ \mathrm{[mJy/beam]}
\end{align*}

with $k_\mathrm{B}$ the Boltzmann-constant,
$T_\mathrm{sys}$ the system temperature,   
$A_\mathrm{eff} = \eta_a A$ the effective aperture of an antenna, assuming apperture efficiency $\eta_a$ at frequency $\nu$,    
$N_\mathrm{p}$ the number of polarisations [$N_\mathrm{p}=2$ for images in Stokes I, Q, U or V],    
$N_\mathrm{a}$ the number of antennas,    
$\Delta\nu$ the RF observation bandwidth [Hz] for continuum or channel width/FWHM of line for spectral line imaging and    
$\Delta t$ the total on target integration time [sec].

The equations are from http://www.atnf.csiro.au/people/Tobias.Westmeier/tools_hihelpers.php, or https://ui.adsabs.harvard.edu/abs/2013tra..book.....W/abstract, respectively.

In [ ]:
# Fitted values from recent work.
specs = np.array([[ 881.           ,24.87730995],
     [ 913.875        ,24.49771763],
     [ 946.75         ,24.16024859],
     [ 979.625        ,23.64646727],
     [1012.5          ,24.07896985],
     [1045.375        ,23.79283849],
     [1078.25         ,22.70843003],
     [1111.125        ,22.93770384],
     [1144.           ,22.84885476],
     [1176.875        ,22.12287765],
     [1209.75         ,21.49206455],
     [1242.625        ,21.16654511],
     [1275.5          ,20.96656328],
     [1308.375        ,20.6466135 ],
     [1341.25         ,20.46467585],
     [1374.125        ,20.35819618],
     [1407.           ,20.33486544],
     [1439.875        ,20.45917325],
     [1472.75         ,20.46422681],
     [1505.625        ,20.53214192],
     [1538.5          ,21.29373981],
     [1571.375        ,20.78716734],
     [1604.25         ,20.91109069],
     [1637.125        ,21.14846713],
     [1670.           ,24.40091906]])
     
f = specs[:,0]*1e6  # frequency axis above in Hz
Tsys_eta = specs[:,1]
obs_freq = 1665.4*1e6  # OH rest frequency in Hz

idx_ = np.argmin(np.abs(f - obs_freq))
# fit with a second degree polynomial
x = f[idx_-1:idx_+2]
y = Tsys_eta[idx_-1:idx_+2]
if len(x) < 3: p_o = 1
else: p_o = 2
c2p = np.polyfit(x, y, p_o) 
p2 = np.poly1d(c2p)
Tsys_per_eta = p2(obs_freq)

The beam-averaged brightness temperature measured by a given array depends on the synthesized beam, and is related to the flux density per beam:
\begin{align*}
 T_\mathrm{B} = \frac{\lambda^2S_\nu}{2k_\mathrm{B}\Omega_\mathrm{bm}}
\end{align*}
\begin{align*}
 \frac{T_\mathrm{B}}{\mathrm{K}} \approx 1.360 \times \left(\frac{\lambda}{\mathrm{cm}} \right)^2 \times \frac{S_\nu}{\mathrm{mJy}} \times \frac{1}{\vartheta^2}
\end{align*}
At OH frequency 1.665 GHz
$\frac{T_\mathrm{B}}{K} \approx 440.633 \times \frac{S_\nu}{\mathrm{mJy}} \times \frac{1}{\vartheta^2}$

\begin{align*}
 T_\mathrm{B} = \frac{441 \, S}{\vartheta^{2}} = F \cdot S
\end{align*}
In this equation, the *HPBW* $\vartheta$ of the synthesized beam is measured in arcseconds, and the flux density $S$ is measured in mJy, to calculate the brightness temperature $T_\mathrm{B}$.

Angular size of a Gaussian beam $\sim \frac{1.5\lambda}{B\mathrm{max}}$.    
The brightness temperature sensitivity can be obtained by substituting the RMS noise, $\sigma_s$, for $S$.

We can then calculate the OH-column density $N_\mathrm{OH}$ by integrating over the OH linewidth (measured in $\mathrm{km}\,\mathrm{s}^{-1}$):
\begin{align*}
N_{\rm OH} = 8.9 \times 10^{12} \! \int \! T_{\rm B} \, \mathrm{d}V          
 \end{align*}

 \begin{align*}
N_{\rm OH} \approx 8.9 \times 10^{12} \cdot (T_{\rm B} \Delta v)          
 \end{align*}
  \begin{align*}
S \approx N_{\rm OH} \ \vartheta^2 \  / \ (8.9 \times 10^{12} * 441 * \Delta v)          
 \end{align*}

In [ ]:
F_tap = 1.6  # The Robust/tapering factor.
theta = 60.  # The synthesized beam HPBW in arcseconds  
SN = 1.5  #5.  # Signal to Noise for detection
N_oh = 4.1e12  # Target OH column Density in cm^{-2}
velocity_width = 3.  # in km/s
f_OH = 1665.4*1e6  # OH rest frequency in Hz
J = 1e26  # Jy to Watts conversion factor
D = 13.5  # m
NPol = 2
A = np.pi * (D/2)**2
N = 58  # nr antennas

col_flux = N_oh*theta**2/(8.9e12*441.*velocity_width)
rmsmin = col_flux/(F_tap*SN)  # mJy/beam

print('Intensity corresponding to an HI column density:\n')
string1 = 'OH'\
            ' with a column density of N_oh = {0:2.4g},\n'\
            '   observed at {1} arcseconds resolution (HPBW),\n'\
            '   with a velocity width of {2:2.4g} km/s,\n'\
            '   is observed with an intensity of {3:2.4g} mJy/beam.\n'
string2 = 'To detect this\n'\
            '   at a {1:.0f}-sigma level (taper factor of {2:2.2g}),\n'\
            '   we require a data cube with a natural rms of {3:2.4g} mJy/beam\n'\
            '   (after regridding to {4:2.4g} km/s-wide channels).\n'
print(string1.format(N_oh, theta, velocity_width, col_flux))
print(string2.format(N_oh, SN, F_tap, rmsmin, velocity_width))

Intensity corresponding to an HI column density:

OH with a column density of N_oh = 4.1e+12,
   observed at 60.0 arcseconds resolution (HPBW),
   with a velocity width of  3 km/s,
   is observed with an intensity of 1.254 mJy/beam.

To detect this
   at a 2-sigma level (taper factor of 1.6),
   we require a data cube with a natural rms of 0.5223 mJy/beam
   (after regridding to  3 km/s-wide channels).



In [ ]:
bw = f_OH/const.c.value*velocity_width*1000.
Tsys_per_eta = Tsys_eta[np.argmin(np.abs(f - obs_freq))]
taumin = np.power((2 * const.k_B.value * Tsys_per_eta * J)/(A * (rmsmin/1000.) * np.sqrt(NPol)),2)/(N * (N-1) * bw)
print ("T_sys/eta at %3.0f MHz is %2.1fK"%(obs_freq/1e6,Tsys_per_eta) )
print('   With this, to reach the required rms level, the required integration time is\n'\
        '   {0:.0f}s or {1:.2f}h.'.format(taumin, taumin/3600.))

T_sys/eta at 1665 MHz is 24.4K
   With this, to reach the required rms level, the required integration time is
   7371s or 2.05h.
